In [4]:
import re
import json
import string
import tqdm
import pandas as pd

from datasets import load_from_disk, Dataset, DatasetDict

with open("./data/wikipedia_documents.json", 'r', encoding='utf-8') as f:
    wiki_json = json.load(f)

wiki_df = pd.DataFrame(list(wiki_json.values()))

with open("./data/KorQuAD/KorQuAD_v1.0_train.json", 'r', encoding='utf-8') as f:
    train_json = json.load(f)

with open("./data/KorQuAD/KorQuAD_v1.0_dev.json", 'r', encoding='utf-8') as f:
    dev_json = json.load(f)

text_json = {**train_json, **dev_json}

In [5]:
wiki_df.keys()

Index(['text', 'corpus_source', 'url', 'domain', 'title', 'author', 'html',
       'document_id'],
      dtype='object')

In [7]:
def flattenList(nested_list):
    flat_list = []
    for sublist in nested_list:
        for item in sublist:
            flat_list.append(item)
    return flat_list

special_characters = re.escape(string.punctuation)

def getOtherCharacters(text):
    text = re.sub(r"[가-힣ㄱ-ㅎㅏ-ㅣA-Za-z0-9一-龥ぁ-ゔァ-ヴー々〆〤 ]", "", text)
    text = re.sub(f'[{special_characters}]', "", text)
    text = re.sub(r"[≪≫《》〈〉＜＞「」『』‘’“”・·°∧­ćä]", "", text)
    text = re.sub(r"[\*\#]+", "",text) # 특수기호 '*', '#'  
    return text

other_characters = [getOtherCharacters(it) for it in wiki_df['text']]
other_characters = set(flattenList(other_characters))
other_characters = (''.join(other_characters))

In [8]:
def removeOtherCharacters(text):
    text = re.sub(f'[{other_characters}]', "", text)
    text = text.replace('\\n', ' ').replace('\n', ' ')
    text = ' '.join(text.split())
    return text

In [9]:
for key in tqdm.tqdm(wiki_json.keys()):
    wiki_json[key]['text'] = removeOtherCharacters(wiki_json[key]['text'])

  0%|          | 0/60613 [00:00<?, ?it/s]

100%|██████████| 60613/60613 [00:08<00:00, 6903.45it/s]


In [10]:
wiki_df = pd.DataFrame(list(wiki_json.values()))
wiki_df.drop_duplicates(subset=['text'], keep='first', inplace=True)
wiki_df.to_json('./data/wikipedia_preprocessed.json', orient='index')